# Intro Scalable Deep Learning: Day 2, Tutorial 2

**Content creators**: Jan Ebert, Stefan Kesselheim, Jenia Jitsev

**Content reviewers / testers**: Stefan Kesselheim, Jenia Jitsev, Mehdi Cherti, Roshni Kamath

**Content supervisors** : Jenia Jitsev

In the first part of this tutorial, you will learn about a powerful tool with regards to handling data pipelines for training. We take a look at converting data to an efficient format, reading it back out, and several preprocessing options to speed up training.

For the second part, you are going to combine what you learned about MPI yesterday with what you just heard about data parallelism to adapt the model you built in the previous tutorial for data parallel distributed training using the [Horovod library](https://horovod.ai/).

## Exercise 1: TensorFlow's Data API

As one of the fundamental challenges in deep learning is wrestling with data, TensorFlow provides a large, functional-style API for handling the different kinds of data we are going to encounter. In an upcoming tutorial, you will learn more about the pipeline. Here, you will first learn a good standard technique to feed data in.

For going with `tf.data` pipelines to _process_ data, TensorFlow provides a binary format to _store_ data called [TFRecord](https://www.tensorflow.org/tutorials/load_data/tfrecord). `tf.data` itself is agnostic to the data container; TFRecord is one option among others (e.g. [LMDB](https://symas.com/lmdb/), [HDF5](https://www.hdfgroup.org/solutions/hdf5), or even a simple ZIP archive). The TFRecord format is based upon serializing and deserializing data using [Protocol Buffers](https://en.wikipedia.org/wiki/Protocol_Buffers) (or "Protobufs") to obtain a format optimized for sequential reading. In case a `tf.data` pipeline takes too much time to process, you may want to consider storing the processed data as a TFRecord dataset.

To get a feel for the TFRecord format, you are now going to convert existing datasets into TFRecords.

### Tasks
1. Convert the CIFAR-10 dataset to TFRecords using the skeleton in `exercise1-1.py`. The code skeleton already takes care of the images; you need to add the treatment for the labels. Try to understand the steps required to go from TensorFlow datatypes to `tf.train.Example` Protobufs.
1. Convert the CIFAR-10 TFRecords back into the original format using `exercise1-2.py` as a basis. Again, the skeleton already reads the images and you need to add the labels.
[The documentation on reading TFRecord files](https://www.tensorflow.org/tutorials/load_data/tfrecord#reading_a_tfrecord_file) may come in handy.

## Exercise 2: Horovod Basics

As it became necessary to scale deep learning, people got the idea to use the well-established principles and mechanisms from MPI (see yesterday) and apply them to deep learning models, thus obtaining simple data parallel distributed training. One library built upon these ideas is called [_Horovod_](https://github.com/horovod/horovod). Building on top of MPI and [NCCL](https://developer.nvidia.com/nccl) (NVIDIA's collective communication library), Horovod gives us a simple interface to convert code running on a single process to code running on many processes. Horovod currently works with the deep learning frameworks TensorFlow, Keras, PyTorch, and MXNet. See the [list of supported frameworks](https://github.com/horovod/horovod#supported-frameworks).

Just as you started learning MPI concepts from a simple 'hello world' script, you will now do the same for Horovod. Similar to `mpi4py`, Horovod provides all primitive methods for getting the rank or world size of the current Horovod process. Sharing another similarity to `mpi4py`, these primitives are not documented online. You can either find them in the [basics module](https://github.com/horovod/horovod/blob/master/horovod/common/basics.py) of Horovod or use Python's built-in `help(function)` command.

Functions like `MPI.Init()`, `comm.Get_rank()`, `comm.Get_size()` have similar equivalents in Horovod – for instance, `hvd.init()`, `hvd.rank()`, and `hvd.size()`. In addition, Horovod workers (or tasks in Slurm jargon) form local groups, with a local rank and a local size for each node. For obtaining those, Horovod offers `hvd.local_rank()` and `hvd.local_size()`.  Workers' assignments happen automatically when using `srun` to start the main training code. Example: if we would like to employ $G=8$ GPUs in total for data parallel distributed training, we may have 2 nodes running 4 worker processes each, with a global size of 8 – the total number of GPUs; the local size for both nodes would be 4 – the number of GPUs taken per node. Global ranks would run from 0 to 7, while local ones would run from 0 to 3 on each node. Horovod's `local_rank` function plays an important role in the proper assignment of a node's local GPUs to each of the node's local worker.

### Tasks

1. Go through the main code in `exercise2.py` and understand the structure.
2. Assign an individual GPU to each worker in `exercise2.py`.
3. Add code to determine the rank, local rank, and world size to the skeleton.
4. Add code to output this information in the `main` function body.
5. Adapt the corresponding job script `exercise2.sh` to contain proper numbers corresponding to the total number of GPUs you would like to run the main code with.
6. Submit the job script. You can experiment with different GPU numbers, different numbers of GPU per node, or different numbers of nodes.

## Exercise 3: From Single- to Multi-Node Distributed Training
In this exercise, you will convert a very simple and clean code example to Horovod. The goal of this tutorial is to obtain a first running code, so the exercise ignores several possible questions.

Let's go through the minimum steps necessary to use Horovod with Keras.

1. Import: `import horovod.tensorflow.keras as hvd`.
1. Initialize Horovod: `hvd.init()` at the beginnig of your script (right after the imports).
1. Assign a local GPU device for each worker. Use `hvd.local_rank()` to figure out proper GPU IDs.
1. Split/shard the dataset.
1. Wrap your optimizer in a `hvd.DistributedOptimizer`:
```python
optimizer = hvd.DistributedOptimizer(optimizer)
```
1. Add the callbacks `hvd.callbacks.BroadcastGlobalVariablesCallback(0)` and `hvd.callbacks.MetricAverageCallback()`. \
The first assures all our workers are correctly synchronized at the start, while the second automatically averages metrics (like the loss) obtained over all processes (while optional, this callback is useful for avoiding bugs later on; it prevents having to worry about properly re-scaling the metrics yourself).
1. Modify code with any kind of produced output or disk access to only use the process with rank 0 (our root process).\
For example, we only want to print the training progress from the root process, and we only want to save checkpoints from the root process so there are no issues with parallel I/O. Root process choice is arbitrary – we decide here for rank 0.

### Tasks
1. Please convert the code in `exercise3.py` by following the pointers above. You are guided through the code by comments. Look out for the word "task".
    - What is the observation you make on training performance? 
    - Bonus: what happens if you tweak the learning rate or batch size?